# Q1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Dropout , Flatten ,MaxPooling2D , BatchNormalization ,experimental
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras.models import Model

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import mnist
from skimage.feature import hog
import tensorflow as tf

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0 # Normalize pixel values to␣
↪[0, 1]

# Convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Load pre-trained VGG-16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32,3)) 

# MNIST images are 28x28, so resize them to 32x32
vgg_model.trainable = False 

# Freeze the weights
def extract_deep_features(images):
print('Extracting deep features')
images = np.expand_dims(images, axis=-1) 

# Add channel dimension for grayscale images
images = np.repeat(images, 3, axis=-1) # Convert grayscale to RGB
images = np.array([tf.image.resize(image, (32, 32)) for image in images])

# Resize images to 32x32
deep_features = vgg_model.predict(images)
return deep_features

def extract_hog_features(images):
    print('Extracting deep features')
    hog_features = []
    for image in images:
    hog_feature = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
    hog_features.append(hog_feature)
    return np.array(hog_features)

# Extract deep features for training and test sets
X_train_deep_features = extract_deep_features(X_train)
X_test_deep_features = extract_deep_features(X_test)

# Extract HOG features for training and test sets
X_train_hog_features = extract_hog_features(X_train)
X_test_hog_features = extract_hog_features(X_test)

# Stack deep features with HOG features
X_train_stacked = np.concatenate((X_train_deep_features.reshape(len(X_train_deep_features), -1), X_train_hog_features), axis=1)
X_test_stacked = np.concatenate((X_test_deep_features.reshape(len(X_test_deep_features), -1), X_test_hog_features), axis=1)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Run the hybrid model 5 times and compute the mean accuracy
accuracies = []
for _ in range(2):
    # Split the data into training and validation sets
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_stacked, y_train, test_size=0.2, random_state=42)
    
    # Train the model
    print('fitting rf model')
    rf_classifier.fit(X_train_split, y_train_split)
    
    # Evaluate the model on the validation set
    val_predictions = rf_classifier.predict(X_val)
    print('getting predictions')
    accuracy = accuracy_score(y_val, val_predictions)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print("Mean accuracy:", mean_accuracy)

# Q2)

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.applications import VGG16
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess MNIST images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Resize MNIST images to match VGG input size
X_train_resized = np.array([cv2.resize(img, (48, 48)) for img in X_train])
X_test_resized = np.array([cv2.resize(img, (48, 48)) for img in X_test])

# Extract deep features from VGG-16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48,3))

def extract_vgg_features(images):
    images = np.repeat(images[..., np.newaxis], 3, -1) # Convert to RGB
    features = vgg_model.predict(images)
    features_flattened = features.reshape(features.shape[0], -1)
    return features_flattened

X_train_vgg_features = extract_vgg_features(X_train_resized)
X_test_vgg_features = extract_vgg_features(X_test_resized)

# Extract SIFT features from MNIST images
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    for img in images:
        img_uint8 = cv2.convertScaleAbs(img) # Convert to CV_8U format
        keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
        if descriptors is not None:
            sift_features.append(descriptors.flatten())
        else:
            sift_features.append(np.zeros(128)) # If no keypoints detected,append zeros
    return np.array(sift_features)

X_train_sift_features = extract_sift_features(X_train)
X_test_sift_features = extract_sift_features(X_test)

# Stack deep features with SIFT features
X_train_stacked = np.hstack((X_train_vgg_features, X_train_sift_features))
X_test_stacked = np.hstack((X_test_vgg_features, X_test_sift_features))

# Train RandomForest classifier
mean_accuracy = 0
for i in range(5):
    # Split data into train and validation sets
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_stacked, y_train, test_size=0.2, random_state=i)
    
    # Train RandomForest classifier
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train_split, y_train_split)
    
    # Predict on validation set
    y_pred = clf.predict(X_val_split)
    
    # Compute accuracy
    accuracy = accuracy_score(y_val_split, y_pred)
    mean_accuracy += accuracy

mean_accuracy /= 5
print("Mean Accuracy:", mean_accuracy)

# Q3)

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.applications import VGG16
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skimage.feature import hog

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess MNIST images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Resize MNIST images to match VGG input size
X_train_resized = np.array([cv2.resize(img, (48, 48)) for img in X_train])
X_test_resized = np.array([cv2.resize(img, (48, 48)) for img in X_test])

# Extract deep features from VGG-16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48,3))

def extract_vgg_features(images):
    images = np.repeat(images[..., np.newaxis], 3, -1) # Convert to RGB
    features = vgg_model.predict(images)
    features_flattened = features.reshape(features.shape[0], -1)
    return features_flattened

X_train_vgg_features = extract_vgg_features(X_train_resized)
X_test_vgg_features = extract_vgg_features(X_test_resized)

# Extract SIFT features from MNIST images
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    for img in images:
        img_uint8 = cv2.convertScaleAbs(img) # Convert to CV_8U format
        keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
        if descriptors is not None:
            sift_features.append(descriptors.flatten())
        else:
            sift_features.append(np.zeros(128)) # If no keypoints detected,append zeros
    return np.array(sift_features)

X_train_sift_features = extract_sift_features(X_train)
X_test_sift_features = extract_sift_features(X_test)

# Extract HOG features from MNIST images
def extract_hog_features(images):
    hog_features = []
    for img in images:
        hog_feat = hog(img, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), block_norm='L2-Hys')
        hog_features.append(hog_feat)
    return np.array(hog_features)

X_train_hog_features = extract_hog_features(X_train)
X_test_hog_features = extract_hog_features(X_test)

# Stack deep features with SIFT and HOG features
X_train_stacked = np.hstack((X_train_vgg_features, X_train_sift_features, X_train_hog_features))
X_test_stacked = np.hstack((X_test_vgg_features, X_test_sift_features, X_test_hog_features))

# Train RandomForest classifier
mean_accuracy = 0
for i in range(5):
    # Split data into train and validation sets
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_stacked, y_train, test_size=0.2, random_state=i)
    
    # Train RandomForest classifier
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train_split, y_train_split)
    
    # Predict on validation set
    y_pred = clf.predict(X_val_split)
    
    # Compute accuracy
    accuracy = accuracy_score(y_val_split, y_pred)
    mean_accuracy += accuracy

mean_accuracy /= 5
print("Mean Accuracy:", mean_accuracy)

# Q4)

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.applications import VGG16
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from sklearn.decomposition import PCA

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess MNIST images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Resize MNIST images to match VGG input size
X_train_resized = np.array([cv2.resize(img, (48, 48)) for img in X_train])
X_test_resized = np.array([cv2.resize(img, (48, 48)) for img in X_test])

# Extract deep features from VGG-16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48,3))

def extract_vgg_features(images):
    images = np.repeat(images[..., np.newaxis], 3, -1) # Convert to RGB
    features = vgg_model.predict(images)
    features_flattened = features.reshape(features.shape[0], -1)
    return features_flattened

X_train_vgg_features = extract_vgg_features(X_train_resized)
X_test_vgg_features = extract_vgg_features(X_test_resized)

# Extract SIFT features from MNIST images
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    for img in images:
        img_uint8 = cv2.convertScaleAbs(img) # Convert to CV_8U format
        keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
        if descriptors is not None:
            sift_features.append(descriptors.flatten())
        else:
            sift_features.append(np.zeros(128)) # If no keypoints detected, append zeros
    return np.array(sift_features)

X_train_sift_features = extract_sift_features(X_train)
X_test_sift_features = extract_sift_features(X_test)

# Extract HOG features from MNIST images
def extract_hog_features(images):
    hog_features = []
    for img in images:
    hog_feat = hog(img, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1), block_norm='L2-Hys')
    hog_features.append(hog_feat)
    return np.array(hog_features)

X_train_hog_features = extract_hog_features(X_train)
X_test_hog_features = extract_hog_features(X_test)

# Stack deep features with SIFT and HOG features
X_train_stacked = np.hstack((X_train_vgg_features, X_train_sift_features, X_train_hog_features))
X_test_stacked = np.hstack((X_test_vgg_features, X_test_sift_features, X_test_hog_features))

# Perform PCA for dimensionality reduction
components_list = [50, 100, 150] # Try different component values
mean_accuracy_list = []

for n_components in components_list:
    mean_accuracy = 0
    for i in range(5):
        # Split data into train and validation sets
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_stacked, y_train, test_size=0.2, random_state=i)

        # Apply PCA
        pca = PCA(n_components=n_components)
        X_train_pca = pca.fit_transform(X_train_split)
        X_val_pca = pca.transform(X_val_split)
        
        # Train RandomForest classifier with hyperparameter tuning
        clf = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=2, random_state=42)
        clf.fit(X_train_pca, y_train_split)
        
        # Predict on validation set
        y_pred = clf.predict(X_val_pca)
        
        # Compute accuracy
        accuracy = accuracy_score(y_val_split, y_pred)
        mean_accuracy += accuracy
        
    mean_accuracy /= 5
    mean_accuracy_list.append(mean_accuracy)
    print(f"Mean Accuracy (PCA Components={n_components}): {mean_accuracy}")
    
# Choose the best component value based on validation performance
best_components = components_list[np.argmax(mean_accuracy_list)]

# Run the model with the best components 5 times and compute the mean accuracy
mean_accuracy_final = 0
for i in range(5):
    # Split data into train and validation sets
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_stacked, y_train, test_size=0.2, random_state=1)
    
    # Apply PCA
    pca = PCA(n_components=best_components)
    X_train_pca = pca.fit_transform(X_train_split)
    X_val_pca = pca.transform(X_val_split)
    
    # Train RandomForest classifier with hyperparameter tuning
    clf = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=2, random_state=42)
    clf.fit(X_train_pca, y_train_split)
    
    # Predict on validation set
    y_pred = clf.predict(X_val_pca)
    
    # Compute accuracy
    accuracy = accuracy_score(y_val_split, y_pred)
    mean_accuracy_final += accuracy

mean_accuracy_final /= 5
print(f"Mean Accuracy (PCA Components={best_components}): {mean_accuracy_final}")